## Ticket System for paying for Transaction Fees

The following code is a PoC of the ticket-system descibed by Vitalik in [this](https://vitalik.ca/general/2023/01/20/stealth.html) blog post.

Essentially, users register a ticket at searchers by sending them small abounts of funds together with a blinded message. The searcher verifies that the funds arrived and then signes the (chaumian) blinded message (blinded ticket) and returns it to the user. The user can then broadcast a transaction not paying any fees (within the transaction). The searcher would pick it up, include it into a block and invalidate the used blinded ticket/signature.

In [1]:
from web3 import Web3
from eth_account import Account
import os
from Crypto.PublicKey import RSA
from Crypto.Util.number import inverse, bytes_to_long, long_to_bytes
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256

web3 = Web3(Web3.HTTPProvider('<YOUR RPC ENDPOINT>'))

skip_broadcasting_tx = True

## CONSTANTS

In [2]:
sender_account = Account.create(os.urandom(32))
sender_private_key = sender_account.privateKey.hex()
sender_address = sender_account.address

# Searcher's address and RSA keys
searcher_address = '0xF0109fC8DF283027b6285cc889F5aA624EaC1F55'
searcher_RSA_private_key = RSA.generate(2048)
searcher_RSA_public_key  = searcher_RSA_private_key.publickey()

print(f'Sender account\nAddress: {sender_address}\nSender private Key: {sender_private_key}\n')
print(f'Searcher address: {searcher_address}')

Sender account
Address: 0x6e06F006CD1eF148960970C546107325123360ee
Sender private Key: 0xdffcd0b452b2a3ebe74ff22f9b394f0b790f54d3f50bba6ca31cac9df3f58b09

Searcher address: 0xF0109fC8DF283027b6285cc889F5aA624EaC1F55


# Sender (wants to register ticket at searcher)

## Generate ticket

In [3]:
# Sender's unblinded ticket
unblinded_ticket = b'This is Tonis ticket #1'

# Hash the unblinded ticket
unblinded_ticket_hash = SHA256.new(unblinded_ticket).digest()

# Convert unblinded ticket hash to a number for RSA operations
unblinded_ticket_long = bytes_to_long(unblinded_ticket_hash)

## Blind ticket

In [4]:
# Choose a random blinding factor and compute the blinded ticket
blinding_factor = bytes_to_long(get_random_bytes(256)) % searcher_RSA_public_key.n
blinded_ticket = (
    unblinded_ticket_long * pow(blinding_factor, searcher_RSA_public_key.e,searcher_RSA_public_key.n)
) % searcher_RSA_public_key.n

print(f'Blinded ticket:\n{web3.toHex(blinded_ticket)}')

Blinded ticket:
0x51fa10bcf797d305cffa74904e1ce89219e7cd1f1a240c5cd6a6f0dd97f048baeca4283c0e04d47a1b689a36d8b19f77a45d1d22aa9d934c71ce3cd5d485ba278a04de31ab9bd4c53290e70e007f75b95bd6d3ba0c07542d3a504fbe08d5dbc6e4c0368ca65c7a2f0387b85122c104e914f749b008b856392c29c4803c8eccaea7c1f903b8034d5fd794c4dd14dc833a48f8095bf7bccc38dc68aa1c0d716013c6e9e559132dbd29edf373d3803b4df7f8a8121a599fa4657787a747b6e21c33dc5452529ec1d6998ebb3eac2398e69cccd3ccfd7cdccf92d444c39eed457f70aed97b7c248ea197ac5a16b5cee8e1f3f0edf9de7ef4a544cff6b1b68996bc70


## Send funds + blinded ticket to searcher

In [5]:
# Details of the transaction
transaction = {
    'to': searcher_address,  
    'value': web3.toWei(0.01, 'ether'),  
    'gas': 21000,
    'gasPrice': web3.toWei('50', 'gwei'),
    'nonce': web3.eth.getTransactionCount(sender_account.address),  
    'chainId': 1,
    'data': web3.toHex(blinded_ticket)
}

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, sender_private_key)

# Get the raw transaction
raw_transaction = signed_txn.rawTransaction

print(f'Signed transaction\n{raw_transaction.hex()}')

Signed transaction
0xf9016d80850ba43b740082520894f0109fc8df283027b6285cc889f5aa624eac1f55872386f26fc10000b9010051fa10bcf797d305cffa74904e1ce89219e7cd1f1a240c5cd6a6f0dd97f048baeca4283c0e04d47a1b689a36d8b19f77a45d1d22aa9d934c71ce3cd5d485ba278a04de31ab9bd4c53290e70e007f75b95bd6d3ba0c07542d3a504fbe08d5dbc6e4c0368ca65c7a2f0387b85122c104e914f749b008b856392c29c4803c8eccaea7c1f903b8034d5fd794c4dd14dc833a48f8095bf7bccc38dc68aa1c0d716013c6e9e559132dbd29edf373d3803b4df7f8a8121a599fa4657787a747b6e21c33dc5452529ec1d6998ebb3eac2398e69cccd3ccfd7cdccf92d444c39eed457f70aed97b7c248ea197ac5a16b5cee8e1f3f0edf9de7ef4a544cff6b1b68996bc7026a08e7f11f5d4eec38b40c4903c4eb797f86d7d31759b0237ddab5944c850ee0f9ca0599d9b653e7d96c92e73459778584ceb2b6d11c907c7e345f8fc0e083195d2e5


The signed transaction can now be broadcasted

In [6]:
# Skip this cell and assumet to have a valid tx hash
if not skip_broadcasting_tx:
    sender_tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)

# Searcher (accepts tickets)

Searcher parses the transaction, verififies the receipt of funds and retrieves the blinded ticket hash

In [7]:
# Skip this cell and assumet to have a valid blinded ticket
if not skip_broadcasting_tx:
    transaction = web3.eth.getTransaction(sender_tx_hash)

    # The 'input' field contains the ticket
    blinded_ticket_hex = transaction['input']

    # Now convert the hexadecimal string back to an integer
    blinded_ticket = int(blinded_ticket_hash_hex, 16)

## Sign blinded ticket

In [8]:
# Searcher signs the blinded message
signed_blinded_ticket = pow(blinded_ticket, searcher_RSA_private_key.d, searcher_RSA_public_key.n)
print(f'Signed blinded ticket\n{web3.toHex(signed_blinded_ticket)}')

Signed blinded ticket
0x40633423943e1da1777b084cb65634f03a5a8c8703bcb08523731fbbd8a4017a3240c909799ee0eae3e556715086ef606a4ac2dbc7358bf4bc628297b61103a4712d25fc5cd31f87a3db5e44ee076f19a075ba878cb565f38df8cb8ff569b77affd6e0dd2f7204fec27c6d42710534cbddd96a0f6722366bc8d5c24ac04f0ef0704b90a208ad03a69bf44b1b7a6162a77e37d485d67d65cd4ef9b42c9327c8e393868c008b0bcb3469fa0319a0dd310abea992b3041135c43b9e390840e3013b2d9abe5ddd259981c334f17ed850a318e7c35f3feff9c1ff7fba345cc025574d6530a66a89e8ad30bbf6020d72f69838d0de8b53b4c2b515bf5fdab3abf4aa0b


Searcher sends the signed blinded ticket to the sender (let's ignore how) - maybe encrypt it with the sender's pubkey and put it into the next built block and let the sender parse those blocks

# Sender 

In [9]:
# Sender unblinds the signed ticket hash
signature = (signed_blinded_ticket * inverse(blinding_factor, searcher_RSA_public_key.n)) % searcher_RSA_public_key.n

# Verify the signature
assert pow(signature, searcher_RSA_public_key.e, searcher_RSA_public_key.n) == unblinded_ticket_long

print("Signature of searcher valid!\n")
print(f'Searcher signature\n{web3.toHex(signature)}')

Signature of searcher valid!

Searcher signature
0x934f59ea90e74a79299da951d705d1235f9002c0c878bd468d5695a3de4e9a5c6a4dd11e7f4c9a226b10e64b1e1221f5af6dcb8261fe23b13a572fed518379c760d206b8ea5235184ba25b3f46cd8cb486390776f2eaf73921e869266b7bce7bc516e8bafe060563f8db25b95e9c7dfb2de9f78a09c01e6470561c9734599061dfe787f714d26d37b2800b01c53f1d4f1358e9ba94c00d668ee1f44fda8485bdb525037cc9686cc3736db8ec9ce981e1d065fa7e3483341d066c816c0fd54e13f8560dbc3019d09c607ee0e4ce3d8b616bcaa4311642072cfc2d011344f3462562d203b2cce362588267849edd6e1a92c42e9f316e76288215c6806dfa4010da


Sender can now priovide the signature to the searcher, proving it has registered a ticket (signature would have to be encrypted using the searchers pubkey or submitted through an private channel).
The searcher would include the transaction in a bundle, although it doesn't pay any feed. In compensation, the searcher would invalidate the used ticket by storing the signature/blinded ticket in a db.

In [10]:
transaction = {
    'to': "0x...",  
    'value': web3.toWei(0.01, 'ether'),  
    'gas': 0,
    'gasPrice': web3.toWei('50', 'gwei'),
    'nonce': web3.eth.getTransactionCount(sender_account.address),  
    'chainId': 1,
    'data': web3.toHex(signature)
}

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, sender_private_key)

# Get the raw transaction
raw_transaction = signed_txn.rawTransaction

print(f'Signed transaction\n{raw_transaction.hex()}')

Signed transaction
0xf9016b80850ba43b74008094f0109fc8df283027b6285cc889f5aa624eac1f55872386f26fc10000b90100934f59ea90e74a79299da951d705d1235f9002c0c878bd468d5695a3de4e9a5c6a4dd11e7f4c9a226b10e64b1e1221f5af6dcb8261fe23b13a572fed518379c760d206b8ea5235184ba25b3f46cd8cb486390776f2eaf73921e869266b7bce7bc516e8bafe060563f8db25b95e9c7dfb2de9f78a09c01e6470561c9734599061dfe787f714d26d37b2800b01c53f1d4f1358e9ba94c00d668ee1f44fda8485bdb525037cc9686cc3736db8ec9ce981e1d065fa7e3483341d066c816c0fd54e13f8560dbc3019d09c607ee0e4ce3d8b616bcaa4311642072cfc2d011344f3462562d203b2cce362588267849edd6e1a92c42e9f316e76288215c6806dfa4010da26a03c47442e94ff80356fa9d291004c26fb1b050fcd82e8386c047fb8acd1a20b42a077956e58d62cbbdffcaa0644208adebad870755c857090bcda3aba0e0ef47a75


Searcher could now include the transaction (although not paying anything) and invalidate the used ticket